In [1]:
import os
os.chdir('..')
from jack import readers
from projects.assertion_mr.shared import AssertionMRPorts
from projects.assertion_mr.qa.definition_model import DefinitionPorts
from jack.io.load import load_squad
from jack.util import preprocessing

/usr/lib64/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import tensorflow as tf
tf.reset_default_graph()
reader = readers.reader_from_file('/home/diwe01/fusessh/r24nlu/triviaqa_wiki/models/100/bilstm_conceptnet_wiki', assertion_dir='/data/assertion_store')


INFO:tensorflow:Restoring parameters from /home/diwe01/fusessh/r24nlu/triviaqa_wiki/models/100/bilstm_conceptnet_wiki/model_module


In [6]:
from jack.io.load import load_jack

dataset = load_jack('../jtr/data/triviaqa/wiki-dev.json')

import json
with open('/run/media/diwe01/Data2/corpora/qa/triviaqa-rc/qa/verified-wikipedia-dev.json') as f:
    verified = json.load(f)


In [9]:
allowed = set()
for d in verified["Data"]:
    allowed.add(d["QuestionId"])

In [10]:
verfied_dataset = [(q, ass) for q, ass in dataset if q.id in allowed]

In [11]:
batch_size = 4

for index in range(0, 1028, batch_size):
    question_batch = [q for q, a in verfied_dataset[index:index + batch_size]]
    answer_batch = [a for q, a in verfied_dataset[index:index + batch_size]]
    
    predicted_answers_wiki = reader(question_batch)
    reader.input_module.use_definitions = False
    predicted_answers = reader(question_batch)
    reader.input_module.use_definitions = True
    for paw, pa, ass, q in zip(predicted_answers_wiki, predicted_answers, answer_batch, question_batch):
        if any(pa[0].text == a.text for a in ass) and all(paw[0].text != a.text for a in ass):
            print("============================= Wrong after Definition ================================")
            print('Question:               ', q.question)
            print('Top answers candidates: ')
            for a in set(a.text for a in pa):
                print('    -', a)
            print()
            print('Answer before Wikipedia:', pa[0].text)
            print('Answer after Wikipedia: ', paw[0].text)
            print()
            print('== Read Wikipedia Definitions ==')
            b = reader.input_module([q])
            rev_vocab = b['__rev_vocab']
            for d, l in zip(b[DefinitionPorts.definitions], b[DefinitionPorts.definition_lengths]):
                print('-', ' '.join(rev_vocab[i] for i in d[:l]))         
            print('=====================================================================================')
            print()
        elif any(paw[0].text == a.text for a in ass) and all(pa[0].text != a.text for a in ass):
            print("=========================== Corrected after Definition ==============================")
            print('Question:               ', q.question)
            print('Top answers candidates: ')
            for a in set(a.text for a in pa):
                print('    -', a)
            print()
            print('Answer before Wikipedia:', pa[0].text)
            print('Answer after Wikipedia: ', paw[0].text)
            print()
            print('== Read Wikipedia Definitions ==')
            b = reader.input_module([q])
            rev_vocab = b['__rev_vocab']
            for d, l in zip(b[DefinitionPorts.definitions], b[DefinitionPorts.definition_lengths]):
                print('-', ' '.join(rev_vocab[i] for i in d[:l]))           
            print('=====================================================================================')
            print()

============================= Wrong after Definition ================================
Question:                Name the game show , presented by Leslie Grantham and Melinda Messenger , where contestants were set physical and mental challenges ?
Top answers candidates: 
    - Cowboy Builders
    - Dirty Den
    - Live from Studio Five
    - Fort Boyard
    - Boyard

Answer before Wikipedia: Fort Boyard
Answer after Wikipedia:  Cowboy Builders

== Read Wikipedia Definitions ==
- Fort Boyard is a French game show created by Jacques Antoine that was first broadcast on 7 July 1990 ( as Les Clés de Fort Boyard , shortened to Fort Boyard from the second series in 1991 ) . It has been re - made around the world over a 25-year period . Fort Boyard is the
- Fort Boyard ( pronounced : [ fɔʁ bɔjaʀ ] ) is a fort located between the Île - d'Aix and the Île d'Oléron in the Pertuis d'Antioche straits , on the west coast of France and is the filming location for the TV gameshow of the same name . Thoug

============================= Wrong after Definition ================================
Question:                In our solar system , the sun is composed of mostly which gas ?
Top answers candidates: 
    - Jupiter
    - helium
    - hydrogen
    - nickel
    - hydrogen and helium . Jupiter and Saturn
    - hydrogen and helium
    - hydrogen and helium . Jupiter

Answer before Wikipedia: hydrogen
Answer after Wikipedia:  hydrogen and helium

== Read Wikipedia Definitions ==
- A hydrogen vehicle is a vehicle that uses hydrogen as its onboard fuel for motive power . Hydrogen vehicles include hydrogen fueled space rockets , as well as automobiles and other transportation vehicles . The power plants of such vehicles convert the chemical energy of hydrogen to mechanical energy either by burning hydrogen
- Hydrogen was a champion Australian thoroughbred racehorse . By the imported stallion Delville Wood ( who also sired Melbourne Cup winner Evening Peal ) he was foaled in 1948 and was trained

- Lolita is a 1997 American - French drama film directed by Adrian Lyne and written by Stephen Schiff . It is the second screen adaptation of Vladimir Nabokov 's novel of the same name and stars Jeremy Irons as Humbert Humbert and Dominique Swain as Dolores \"Lolita\ " Haze , with supporting roles by Melanie Griffith
- Lolita is a 1962 British - American black comedy - drama film directed by Stanley Kubrick based on the novel of the same title by Vladimir Nabokov , about a middle - aged man who becomes obsessed with a teenage girl . The film stars James Mason as Humbert Humbert , Sue Lyon as Dolores Haze ( Lolita ) , and Shelley
- Lolita is a novel by Vladimir Nabokov , written in English and published in 1955 in Paris , in 1958 in New York City , and in 1959 in London . Nabokov 's own translation of the book into Russian was published by Phaedra Publishers in New York in 1967 . The novel is notable for
- Reading Lolita in Tehran : A Memoir in Books is a book by Iranian author and prof

============================= Wrong after Definition ================================
Question:                Where were the FIBA ( Basketball ) World Championships held in 2002 ?
Top answers candidates: 
    - Germany
    - Yugoslavia
    - Indianapolis
    - Yugoslavia and FR Yugoslavia
    - USA Basketball and Indianapolis

Answer before Wikipedia: Indianapolis
Answer after Wikipedia:  Yugoslavia

== Read Wikipedia Definitions ==
- The Indianapolis Motor Speedway , located in Speedway , Indiana , ( an enclave suburb of Indianapolis ) in the United States , is the home of the Indianapolis 500 and the Brickyard 400 . It is located on the corner of 16th Street and Georgetown Road , approximately six miles ( 10 km ) west of Downtown Indianapolis . Constructed
- Indianapolis International Airport ( IATA : IND , ICAO : KIND , FAA LID : IND ) is a public airport seven miles ( 11 km ) southwest of downtown Indianapolis , in Marion County , Indiana , United States . It is owned and operated

=========================== Corrected after Definition ==============================
Question:                What NZ thoroughbred racehorse , foaled in 1925 by the brood mare Marsa and the sire of Phar Lap , was the first horse to win both the Melbourne Cup and Cox Plate in the same year ?
Top answers candidates: 
    - Fields of Omagh
    - Windbag
    - old Manfred
    - Manfred
    - Nightmarch
    - Daryl ' s Joy
    - Makybe Diva
    - Makybe Diva , Might and Power

Answer before Wikipedia: Manfred
Answer after Wikipedia:  Nightmarch

== Read Wikipedia Definitions ==
- Makybe Diva is a British - bred , Australian - trained Thoroughbred racehorse who became the first horse to win the Melbourne Cup on three occasions ( 2003 , 2004 , and 2005 ) . In 2005 , she also won the Cox Plate . Makybe Diva is the highest stakes - earner in Australasian horse racing history , finishing with winnings of more than
- Nightmarch , foaled in 1925 was an outstanding New Zealand bred Thoroughbred ra

============================= Wrong after Definition ================================
Question:                Which modern musical instrument was developed from the shawm ?
Top answers candidates: 
    - oboe
    - double reed
    - bassoon
    - trumpet
    - double - reed
    - reed

Answer before Wikipedia: oboe
Answer after Wikipedia:  trumpet

== Read Wikipedia Definitions ==
- Oboe was a British aerial blind bombing targeting system in World War II , based on radio transponder technology . The system consisted of a pair of radio transmitters on the ground , which sent signals which were received and retransmitted by a transponder in the aircraft . By comparing the time each signal
- Oboes /ˈoʊboʊ/ OH - boh are a family of double reed woodwind musical instruments . The most common oboe plays in the treble or soprano range . Oboes are usually made of wood , but there are also oboes made of synthetic materials . A soprano oboe measures roughly 65 cm ( 25 1⁄2 in ) long , with
- A re

============================= Wrong after Definition ================================
Question:                Canaletto is famous for his landscapes of Venice and which other city ?
Top answers candidates: 
    - Antwerp
    - Carlisle
    - London
    - Milan

Answer before Wikipedia: London
Answer after Wikipedia:  Carlisle

== Read Wikipedia Definitions ==
- London is a city located in Southwestern Ontario , Canada along the Quebec City – Windsor Corridor . The city has a population of 366,151 according to the 2011 Canadian census . London is at the confluence of the non - navigable Thames River , approximately halfway between Toronto , Ontario and Detroit , Michigan . The City of London is
- London Records is a record label in the U.K. that marketed records in the U.S , Canada , and Latin America from 1947 to 1979 before becoming semi - independent .
- London /ˈlʌndən/ is the capital and most populous city of England and the United Kingdom . Standing on the River Thames in the sou

============================= Wrong after Definition ================================
Question:                Which 1800 battle saw Napoleon ' s forces victorious over the Austrians near Alessandria in Piedmont ?
Top answers candidates: 
    - Marengo Battle
    - Marengo
    - Battle of
    - Wagram
    - Battle of Marengo
    - Marengo on 14
    - Marengo on 14 June

Answer before Wikipedia: Marengo
Answer after Wikipedia:  Battle of Marengo

== Read Wikipedia Definitions ==
- The Battle of Marengo was fought on 14 June 1800 between French forces under Napoleon Bonaparte and Austrian forces near the city of Alessandria , in Piedmont , Italy . Near the end of the day , the French overcame General Michael von Melas 's surprise attack , driving the Austrians out of Italy , and enhancing
- Marengo County is a county of the U.S. state of Alabama . As of the 2010 census , the population was 21,027 . The largest city is Demopolis and the county seat is Linden . It is named in honor of the 

- Brier is a city in Snohomish County , Washington , United States . It is bordered by Mountlake Terrace to the west , Lynnwood to the north , Bothell to the east , and the Lake Forest Park to the south . The population was 6,087 at the 2010 census .
- The Tim Hortons Brier , or simply ( and more commonly ) the Brier , is the annual Canadian men 's curling championship , sanctioned by the Canadian Curling Association ( CCA ) . The current event name refers to its main sponsor , the Tim Hortons coffee and doughnut shop chain . \"Brier\ " originally referred to a brand of tobacco
- Windermere is the largest natural lake in England . It is a ribbon lake formed in a glacial trough after the retreat of ice at the start of the current interglacial period . It has been one of the country 's most popular places for holidays and summer homes since the arrival of

=========================== Corrected after Definition ==============================
Question:                What was singer Jack J

InvalidArgumentError: ConcatOp : Expected concatenating dimensions in the range [0, 0), but got 0
	 [[Node: bilstm_assertion_reader/answer_layer/cond/segment_top_k/concat = ConcatV2[N=2, T=DT_INT64, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](bilstm_assertion_reader/answer_layer/cond/segment_top_k/Squeeze, bilstm_assertion_reader/answer_layer/cond/segment_top_k/sub_1, bilstm_assertion_reader/answer_layer/cond/range/start)]]

Caused by op 'bilstm_assertion_reader/answer_layer/cond/segment_top_k/concat', defined at:
  File "/usr/lib64/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib64/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/python3.6/s

In [12]:
len(verfied_dataset)

318

In [5]:
for q, ass in dataset:
    if q.question.startswith("Beloved "):
        print(q.question)
        for a in ass:
            print(a.text)

Beloved in 1999 was whose first movie since The Color Purple in 1985 ?
Oprah
Oprah Winfrey
Oprah
Oprah Winfrey
Oprah
Oprah Winfrey
Oprah
Oprah
Oprah
Oprah Winfrey
Oprah
Oprah Winfrey
Oprah
Oprah Winfrey
Oprah
Oprah Winfrey


In [1]:
import os
os.chdir('..')
from projects.assertion_mr.assertions.store import AssertionStore

In [2]:
s = AssertionStore('/data/assertion_store')

In [6]:
for e in s.assertion_keys_for_subject('chuck yeager', 'wikipedia_firstsent'):
    print(s.get_assertion(e))

Charles Elwood \"Chuck\" Yeager (/ˈjeɪɡər/; born February 13, 1923) is a former United States Air Force officer and record-setting test pilot. In 1947, he became the first pilot confirmed to have exceeded the speed of sound in level flight. Yeager's career began in World War II as a private in
Y (named wye /ˈwaɪ/, plural wyes) is the 25th and penultimate letter in the modern English alphabet and the ISO basic Latin alphabet. In the English writing system, it sometimes represents a vowel and sometimes a consonant.
C is the third letter in the English alphabet and a letter of the alphabets of many other writing systems which inherited it from the Latin alphabet. It is also the third letter of the ISO basic Latin alphabet. It is named cee (pronounced /ˈsiː/) in English.


In [4]:
import re
from bz2 import BZ2File
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor

def myopen(fn):
    return BZ2File(fn) if fn.endswith('.bz2') else open(fn)


def uncamel(name):
    return re.sub('([a-z0-9])([A-Z])', r'\1 \2', name)


def process_line(l):
    l = l.decode('utf-8')
    if l.startswith('#'):
        return None, None
    split = l.split('> ')
    subj = split[0][split[0].rindex('/') + 1:]
    if subj.startswith('List_of') or '(disambiguation)' in subj or '__' in subj:
        return None, None
    obj = split[2]
    if obj.startswith('"'):
        obj = obj[1:obj.find('"@en')]
    else:
        obj = obj[obj.rindex('/') + 1:]
    return subj, obj

    
transitive_redirects = dict()
with myopen('/home/diwe01/Downloads/transitive_redirects_en.ttl.bz2') as f:
    ct = 0
    for subj, obj in map(process_line, f):
        if subj is not None and not (obj.startswith('List_of') or '(disambiguation)' in obj or '__' in obj):
            transitive_redirects[subj] = obj
        ct += 1
        if ct % 1000000 == 0:
            print('%d lines processed...' % ct)

1000000 lines processed...
2000000 lines processed...
3000000 lines processed...
4000000 lines processed...
5000000 lines processed...
6000000 lines processed...
7000000 lines processed...


In [21]:
with myopen('/home/diwe01/Downloads/anchor_text_en.ttl.bz2') as f:
    anchor2articles = defaultdict(lambda: defaultdict(int))
    ct = 0
    for article, anchor_text in map(process_line, f):
        if article is not None:
            anchor2articles[anchor_text.lower()][transitive_redirects.get(article, article)] += 1
        ct += 1
        if ct % 1000000 == 0:
            print('%d lines processed...' % ct)

1000000 lines processed...
2000000 lines processed...
3000000 lines processed...


KeyboardInterrupt: 

In [16]:
labels = defaultdict(set)
for anchor, articles in anchor2articles.items():
    articles = sorted([(a, ct) for a, ct in articles.items() if ct > 10],
                      key=lambda x: -x[1])[:3]
    for a, _ in articles:
        labels[a].add(anchor)

In [20]:
labels['West_End_theatre']

{'london', 'west end', 'west end theatre'}

In [ ]:
labels['london']

In [9]:
batch_size=32
batches = reader.input_module.batch_generator(dataset, batch_size, is_eval=True)
ct = 0
for batch in batches:
    l = max(batch[AssertionMRPorts.support_length])
    ct += 1
    if ct % 10 == 0:
        print(ct * batch_size)
    if l > 600:
        break
print('done')

320
640
960
1280
1600
1920
2240


KeyboardInterrupt: 

In [11]:
batch

{<TensorPort (answer2support)>: array([ 0,  0,  1,  1,  4,  4,  4,  4,  5,  5,  5,  6,  6,  6,  6,  6,  8,
         8,  9,  9,  9, 10, 11, 12, 12, 14, 14, 14, 16, 18, 20, 21, 22, 23,
        23, 23, 23, 29, 33, 33, 33, 33, 33, 33, 34, 34, 34, 35, 37, 37, 39,
        40, 40, 41, 41, 44, 44, 44, 44, 44, 44, 49, 51, 51, 51, 52, 52, 52,
        53, 53, 53, 54, 54, 55, 55, 55, 55, 56, 56, 57, 57, 58, 59, 63, 63,
        65, 68, 71, 74, 80, 84, 84, 84, 84, 85, 85, 85, 85, 85, 86, 86, 86,
        87, 87]), <TensorPort (answer_span)>: array([[ 12,  12],
        [ 77,  77],
        [  9,   9],
        [ 23,  23],
        [ 33,  33],
        [ 50,  50],
        [407, 407],
        [419, 419],
        [201, 201],
        [171, 171],
        [486, 486],
        [ 63,  63],
        [319, 319],
        [272, 272],
        [479, 479],
        [568, 568],
        [153, 153],
        [204, 204],
        [ 75,  75],
        [142, 142],
        [149, 149],
        [ 88,  88],
        [ 63,  63],
        